In [1]:
import mujoco_py
import os
import numpy as np
from mujoco_dynamics import MujocoDynamics
import multiprocessing as mp
import time

Choosing the latest nvidia driver: /usr/lib/nvidia-418, among ['/usr/lib/nvidia-375', '/usr/lib/nvidia-418']
Choosing the latest nvidia driver: /usr/lib/nvidia-418, among ['/usr/lib/nvidia-375', '/usr/lib/nvidia-418']


In [2]:
import mujoco_dynamics
xml_path = os.path.join(os.getcwd(), 'xmls', 'half_cheetah.xml')
dynamics2 = mujoco_dynamics.MujocoDynamics(xml_path, x_eps = 1e-6, u_eps = 1e-6)

time0 = time.time()
for i in range(1000):
    
    dynamics2.set_state(np.random.uniform(-1.0, 1.0, (dynamics2.state_size,)))
    dynamics2.f_x(dynamics2.get_state(), np.array([0.0]))
print("time", time.time() - time0)

time 0.5453927516937256


In [3]:
import os
import numpy as np
import mujoco_dynamics
dynamics3 = mujoco_dynamics.MujocoDynamics(xml_path, x_eps = 1e-6, u_eps = 13-6)

In [4]:
import mujoco_py as mj
from mujoco_py_deriv import MjDerivative
sim = dynamics3.sim
dmain = sim.data
model = sim.model
deriv_obj1 = MjDerivative(model, dmain, ["qacc"], ["qpos"], nwarmup = 1, niter = 1)
deriv_obj2 = MjDerivative(model, dmain, ["qacc"], ["qvel"], nwarmup = 1, niter = 1)

time0 = time.time()
for i in range(1000):
    
    #time0 = time.time()
    dynamics3.set_state(np.random.uniform(-1.0, 1.0, (dynamics3.state_size,)))
    
    deriv1 = deriv_obj1.compute()
    deriv2 = deriv_obj2.compute()
    dqacc_dqpos = deriv1[0][0]
    dqacc_dqvel = deriv2[0][0]
    
    #time1 = time.time()
    
    f_x = np.eye(dynamics3.state_size)
    f_x[:dynamics3.state_size//2, :dynamics3.state_size//2] += 0.5 * (dynamics3.dt ** 2) * dqacc_dqpos
    f_x[:dynamics3.state_size//2, dynamics3.state_size//2:] += np.eye(dynamics3.state_size // 2) * dynamics3.dt
    f_x[:dynamics3.state_size//2, dynamics3.state_size//2:] += 0.5 * (dynamics3.dt ** 2) * dqacc_dqvel

    f_x[dynamics3.state_size//2:, :dynamics3.state_size//2] += dynamics3.dt * dqacc_dqpos
    f_x[dynamics3.state_size//2:, dynamics3.state_size//2:] += dynamics3.dt * dqacc_dqvel
    
    #time2 = time.time()
    #print("time0-1:", time1-time0, "time1-2", time2-time1)

print("time", time.time() - time0)

time 0.6982851028442383


In [151]:
import mujoco_py as mj
from mujoco_py_deriv import MjDerivative, checkderiv

# Prepare mujoco model and data

print(dmain.qvel)
# To compute δf/δx
f = ["qacc"]
x = ["qvel"]
deriv_obj1 = MjDerivative(model, dmain, f, x)
deriv1 = deriv_obj1.compute()
print(deriv1)

deriv_obj2 = MjDerivative(model, dmain, f, x, nwarmup = 3, niter = 10)
deriv2 = deriv_obj2.compute()
print(deriv2 - deriv1)

[ 0.8308427   0.40764689 -0.63954874 -0.56078832  0.11812803  0.47434226
 -0.30082169 -0.41879214  0.83500044]
[[[[-4.33505044e+01  3.60605692e+01  2.86849559e+00 -3.84500967e+00
     9.65172489e-01  1.63424829e-05 -3.78360494e+00 -3.11016188e+00
    -5.68473666e-01]
   [ 3.63277109e+01 -9.30943863e+01  1.12328155e+01 -2.03465311e+00
     7.34696528e-01  2.81113444e-02 -1.20575305e+00 -8.40982921e-01
    -1.81204314e-02]
   [ 3.70921753e+01  8.07788114e+01 -3.46446961e+01 -5.47668935e+00
     2.68965243e+00  1.59933506e-01  5.54184544e+00  7.70669362e+00
     1.37799532e+00]
   [-1.77060866e+01  2.23789872e+01 -8.90929300e-01 -1.10059246e+02
     6.64813115e-01  3.83621767e-01 -1.16296067e+00 -8.80384079e-01
    -1.71644984e-01]
   [-3.34431882e+01 -1.03608069e+01  2.04581297e+01  3.32018307e+01
    -2.05206539e+01  2.87761452e+00  8.60026717e-01 -3.22672668e+00
    -7.54870939e-01]
   [-4.12448447e+01  4.55005479e+01  1.01555531e+00  2.41503697e+01
     3.71142573e+00 -2.68645504e+01 

x(i+1) = x(i) + v(i) * dt + 0.5 a(i) * dt^2

v(i+1) = v(i) + a(i) * dt

In [89]:
import os
import numpy as np
import mujoco_py as mj
from mujoco_py_deriv import MjDerivative, checkderiv


center = np.zeros_like(sim.data.qvel)
center[:] = dmain.qvel
sim.data.qvel[:] = center
print(sim.data.qvel)
sim.forward()
qacc_center = np.zeros_like(sim.data.qacc)
qacc_center[:] = sim.data.qacc
deriv = np.zeros((sim.data.qacc.shape[0], sim.data.qvel.shape[0]))
for i in range(center.shape[0]):
    center[i] += 1e-6
    sim.data.qvel[:] = center
    sim.forward()
    deriv[:, i] = (sim.data.qacc - qacc_center) / 1e-6
    center[i] -= 1e-6
print(deriv)

[-0.727485    0.88537822 -0.9781251  -0.32160277]
[[-9.98890248e-01  3.89653532e-04  0.00000000e+00  0.00000000e+00]
 [ 3.83875043e-04 -9.99827227e-01  1.11022302e-10  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -1.00000000e+02  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.00000000e+02]]


In [3]:
import os
import mujoco_dynamics
xml_path = os.path.join(os.getcwd(), 'xmls', 'swimmer.xml')
dynamics = mujoco_dynamics.MujocoDynamics(xml_path, x_eps = 1e-6, u_eps = 1e-6)
print(dynamics.state_size)

10


In [4]:
from mujoco_py import MjViewer
import numpy as np
viewer = MjViewer(dynamics.sim)
for i in range(100):
    state = dynamics.get_state()
    state[2] = 0
    state[3] = np.pi / 6
    state[4] = - np.pi / 6
    dynamics.set_state(state)
    viewer.render()

Creating window glfw
